In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px 
pd.options.plotting.backend = 'plotly' 
pd.set_option('display.max_columns', None)

# install tortoise

In [1]:
pip install tortoise-tts

     ---------------------------------------- 0.0/29.9 MB ? eta -:--:--
     --------------------------------------- 0.0/29.9 MB 660.6 kB/s eta 0:00:46
     ---------------------------------------- 0.2/29.9 MB 2.0 MB/s eta 0:00:16
     - -------------------------------------- 1.1/29.9 MB 8.9 MB/s eta 0:00:04
     -- ------------------------------------- 2.1/29.9 MB 14.8 MB/s eta 0:00:02
     -- ------------------------------------- 2.1/29.9 MB 14.8 MB/s eta 0:00:02
     -- ------------------------------------- 2.1/29.9 MB 14.8 MB/s eta 0:00:02
     -- ------------------------------------- 2.1/29.9 MB 14.8 MB/s eta 0:00:02
     -- ------------------------------------- 2.1/29.9 MB 14.8 MB/s eta 0:00:02
     ----- ---------------------------------- 4.2/29.9 MB 10.3 MB/s eta 0:00:03
     ------- -------------------------------- 5.4/29.9 MB 12.0 MB/s eta 0:00:03
     --------- ------------------------------ 7.3/29.9 MB 14.6 MB/s eta 0:00:02
     ----------- ---------------------------- 8.6/

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires transformers>=4.33.0, but you have transformers 4.31.0 which is incompatible.


NameError: name 'clips_paths' is not defined

In [2]:
import argparse
import os
import sys
import tempfile
import time

import torch
import torchaudio

from tortoise.api import MODELS_DIR, TextToSpeech
from tortoise.utils.audio import get_voices, load_voices, load_audio
from tortoise.utils.text import split_and_recombine_text

d:\Python\Install\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
#tts = TextToSpeech()
# If you want to use deepspeed the pass use_deepspeed=True nearly 2x faster than normal
#tts = TextToSpeech(use_deepspeed=True, kv_cache=True)

In [5]:
pip install deepspeed


  Using cached deepspeed-0.13.1.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\BOssi\AppData\Local\Temp\pip-install-5kf1r_37\deepspeed_69ed4c45c35144bd94314d4708006ead\setup.py", line 100, in <module>
          cuda_major_ver, cuda_minor_ver = installed_cuda_version()
                                           ^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\BOssi\AppData\Local\Temp\pip-install-5kf1r_37\deepspeed_69ed4c45c35144bd94314d4708006ead\op_builder\builder.py", line 50, in installed_cuda_version
          raise MissingCUDAException("CUDA_HOME does not exist, unable to compile CUDA op(s)")
      op_builder.builder.MissingCUDAException: CUDA_HOME does not exist, unable to compile CUDA op(s)
      [end of output]
  
  note: This error originates from a sub

In [4]:
tts = TextToSpeech()
#tts = TextToSpeech(use_deepspeed=True, kv_cache=True)

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

In [5]:
# This is the text that will be spoken.
text = "Iron Man is a superhero appearing in American comic books published by Marvel Comics."

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [6]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

#Audio('tortoise/voices/tom/1.wav')

Invalid switch - "voices".


In [7]:
%time
text

CPU times: total: 0 ns
Wall time: 0 ns


'Iron Man is a superhero appearing in American comic books published by Marvel Comics.'

In [12]:
%time
# Pick one of the voices from the output above
voice = 'tom'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

Generating autoregressive samples..


100%|██████████| 24/24 [03:40<00:00,  9.18s/it]


Computing best candidates using CLVP


100%|██████████| 24/24 [00:03<00:00,  7.56it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:05<00:00, 13.85it/s]


In [10]:
len(text)

85

In [14]:
(85 /41823) *100

0.20323745307605862

In [13]:
# if processing .2% of the total text takes 3.5 minutes, then processing the entire text will take 29.16 hours

3.5070140280561124

In [8]:
%time
# Pick one of the voices from the output above
voice = 'train_atkins'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

CPU times: total: 0 ns
Wall time: 0 ns
Generating autoregressive samples..


100%|██████████| 24/24 [03:20<00:00,  8.35s/it]


Computing best candidates using CLVP


100%|██████████| 24/24 [00:03<00:00,  7.19it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:05<00:00, 13.73it/s]


In [ ]:
%time
# Pick one of the voices from the output above
voice = 'train_atkins'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [13]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("Iron Man is a superhero appearing in American comic books published by Marvel Comics.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

rlg_auto.pth: 100%|██████████| 25.2M/25.2M [00:00<00:00, 54.1MB/s]
d:\Python\Install\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BOssi\.cache\tortoise\models\models--Manmay--tortoise-tts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
rlg_diffuser.pth: 100%|██████████| 101M/101M [00:00<00:00, 1

Generating autoregressive samples..


100%|██████████| 24/24 [02:55<00:00,  7.30s/it]


Computing best candidates using CLVP


100%|██████████| 24/24 [00:03<00:00,  6.76it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:05<00:00, 13.97it/s]
